# Sarcasm Detection
 **Acknowledgement**

Misra, Rishabh, and Prahal Arora. "Sarcasm Detection using Hybrid Neural Network." arXiv preprint arXiv:1908.07414 (2019).

**Required Files given in below link.**

https://drive.google.com/drive/folders/1xUnF35naPGU63xwRDVGc-DkZ3M8V5mMk

## Install `Tensorflow2.0` 

## Get Required Files from Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
import warnings
import tensorflow as tf
import pickle
from tensorflow.keras import layers
from tensorflow.keras import preprocessing
import numpy as np
import pandas as pd
import json
from sklearn.model_selection import train_test_split
import pprint
from tensorflow.keras.layers import Bidirectional,LSTM,Dense,Dropout,BatchNormalization,Flatten,Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import concatenate
from numpy import array
import nltk
import re
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [3]:
tf.__version__

'2.2.0'

#**## Reading and Exploring Data**

## Read Data "Sarcasm_Headlines_Dataset.json". Explore the data and get  some insights about the data. ( 4 marks)
Hint - As its in json format you need to use pandas.read_json function. Give paraemeter lines = True.

In [4]:
df=pd.read_json("/content/drive/My Drive/Sarcasm Detection/Data/Sarcasm_Headlines_Dataset.json",lines=True )

In [5]:
df.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


## Drop `article_link` from dataset. ( 2 marks)
As we only need headline text data and is_sarcastic column for this project. We can drop artical link column here.

In [6]:
df1=df.drop(labels='article_link', axis=1,inplace=True)


In [7]:
df.head(2)

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0


In [8]:
df.shape

(26709, 2)

In [9]:
df.dtypes

headline        object
is_sarcastic     int64
dtype: object

In [10]:
df['is_sarcastic'].value_counts()

0    14985
1    11724
Name: is_sarcastic, dtype: int64

## Get the Length of each line and find the maximum length. ( 4 marks)
As different lines are of different length. We need to pad the our sequences using the max length.

In [11]:
headline_max_length=max(len(txt) for txt in df['headline'])
headline_max_length

254

In [12]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(df['headline']) #Fit it on Headlines
input_seq = tokenizer.texts_to_sequences(df['headline'])
input_seq[0:2] #Display some converted sentences

[[307, 15114, 678, 3336, 2297, 47, 381, 2575, 15115, 5, 2576, 8433],
 [3, 8434, 3337, 2745, 21, 1, 165, 8435, 415, 3111, 5, 257, 8, 1001]]

In [13]:
vocab_size = len(tokenizer.word_index)
print(vocab_size)

29656


In [14]:
input_seq = tf.keras.preprocessing.sequence.pad_sequences(input_seq, 
                                                                   maxlen=headline_max_length,
                                                                   padding='pre')
print(input_seq.shape)

(26709, 254)


#**## Modelling**

## Import required modules required for modelling.

In [15]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential

# Set Different Parameters for the model. ( 2 marks)

In [16]:
max_features = 10000
maxlen = headline_max_length
embedding_size = 200

## Apply Keras Tokenizer of headline column of your data.  ( 4 marks)
Hint - First create a tokenizer instance using Tokenizer(num_words=max_features) 
And then fit this tokenizer instance on your data column df['headline'] using .fit_on_texts()

# Define X and y for your model.

In [17]:
X = tokenizer.texts_to_sequences(df['headline'])
X = pad_sequences(X, maxlen = maxlen)
y = np.asarray(df['is_sarcastic'])

print("Number of Samples:", len(X))
print(X[0])
print("Number of Labels: ", len(y))
print(y[0])

Number of Samples: 26709
[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0  

In [18]:
X=input_seq
y=np.asarray(df['is_sarcastic'])


In [19]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size=0.2, 
    random_state=42
)

In [20]:
y_val.shape

(5342,)

In [21]:
X_train.shape

(21367, 254)

## Get the Vocabulary size ( 2 marks)
Hint : You can use tokenizer.word_index.

In [22]:
vocab_size = len(tokenizer.word_index)
print(vocab_size)

29656


#**## Word Embedding**

## Get Glove Word Embeddings

In [25]:
glove_file = "/content/drive/My Drive/Sarcasm Detection/Data/glove.6B.zip"

In [26]:
#Extract Glove embedding zip file
from zipfile import ZipFile
with ZipFile(glove_file, 'r') as z:
  z.extractall()

# Get the Word Embeddings using Embedding file as given below.

In [27]:
EMBEDDING_FILE = './glove.6B.200d.txt'

embeddings = {}
for o in open(EMBEDDING_FILE):
    word = o.split(" ")[0]
    # print(word)
    embd = o.split(" ")[1:]
    embd = np.asarray(embd, dtype='float32')
    # print(embd)
    embeddings[word] = embd



# Create a weight matrix for words in training docs

In [28]:
embedding_matrix = np.zeros((vocab_size+1, 200))

for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

len(embeddings.values())

400000

## Create and Compile your Model  ( 7 marks)
Hint - Use Sequential model instance and then add Embedding layer, Bidirectional(LSTM) layer, then dense and dropout layers as required. 
In the end add a final dense layer with sigmoid activation for binary classification.


In [ ]:
### Embedding layer for hint 
## model.add(Embedding(num_words, embedding_size, weights = [embedding_matrix]))
### Bidirectional LSTM layer for hint 
## model.add(Bidirectional(LSTM(128, return_sequences = True)))

In [29]:
model = tf.keras.Sequential()

In [30]:
model.add(tf.keras.layers.Embedding(input_dim=vocab_size+1,
                  output_dim=embedding_size,
                  weights=[embedding_matrix],
                  trainable=False, mask_zero= True))

In [31]:
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Bidirectional(LSTM(units=128,return_sequences=False),merge_mode='concat'))


In [32]:
model.output

<tf.Tensor 'bidirectional/Identity:0' shape=(None, 256) dtype=float32>

In [33]:
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

In [34]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [35]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 200)         5931400   
_________________________________________________________________
dropout (Dropout)            (None, None, 200)         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               336896    
_________________________________________________________________
dense (Dense)                (None, 1)                 257       
Total params: 6,268,553
Trainable params: 337,153
Non-trainable params: 5,931,400
_________________________________________________________________


# Fit your model with a batch size of 100 and validation_split = 0.2. and state the validation accuracy ( 5 marks)


In [36]:
batch_size = 100
epochs = 5

## Add your code here ##

In [37]:
model.fit(X_train,y_train,
          epochs=5,
          batch_size=100, validation_data=(X_val, y_val))

Epoch 1/5
214/214 [==============================] - 215s 1s/step - loss: 0.4818 - accuracy: 0.7676 - val_loss: 0.4069 - val_accuracy: 0.8134
Epoch 2/5
214/214 [==============================] - 213s 997ms/step - loss: 0.3584 - accuracy: 0.8395 - val_loss: 0.3372 - val_accuracy: 0.8529
Epoch 3/5
214/214 [==============================] - 212s 993ms/step - loss: 0.2956 - accuracy: 0.8731 - val_loss: 0.3124 - val_accuracy: 0.8673
Epoch 4/5
214/214 [==============================] - 213s 997ms/step - loss: 0.2561 - accuracy: 0.8919 - val_loss: 0.3119 - val_accuracy: 0.8622
Epoch 5/5
214/214 [==============================] - 213s 993ms/step - loss: 0.2241 - accuracy: 0.9093 - val_loss: 0.3037 - val_accuracy: 0.8735


Validation Accuracy with Embedding size of 200 and RNN Units of 128 is 87.35

# Model 2- Glove Embedding of 300, With 256 Memory Size

In [38]:
EMBEDDING_FILE = './glove.6B.300d.txt'

embeddings = {}
for o in open(EMBEDDING_FILE):
    word = o.split(" ")[0]
    # print(word)
    embd = o.split(" ")[1:]
    embd = np.asarray(embd, dtype='float32')
    # print(embd)
    embeddings[word] = embd

In [39]:
embedding_matrix = np.zeros((vocab_size+1, 300))

for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

len(embeddings.values())

400000

In [40]:
model2 = tf.keras.Sequential()
model2.add(tf.keras.layers.Embedding(input_dim=vocab_size+1,
                  output_dim=300,
                  weights=[embedding_matrix],
                  trainable=False, mask_zero= True))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(tf.keras.layers.Bidirectional(LSTM(units=256,return_sequences=False),merge_mode='concat'))
model2.add(tf.keras.layers.Dense(1,activation='sigmoid'))
model2.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [41]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 300)         8897100   
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 300)         0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 512)               1140736   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 10,038,349
Trainable params: 1,141,249
Non-trainable params: 8,897,100
_________________________________________________________________


In [42]:
model2.fit(X_train,y_train,
          epochs=5,
          batch_size=100, validation_data=(X_val, y_val))

Epoch 1/5
214/214 [==============================] - 219s 1s/step - loss: 0.4574 - accuracy: 0.7815 - val_loss: 0.3763 - val_accuracy: 0.8297
Epoch 2/5
214/214 [==============================] - 217s 1s/step - loss: 0.3308 - accuracy: 0.8523 - val_loss: 0.3276 - val_accuracy: 0.8504
Epoch 3/5
214/214 [==============================] - 218s 1s/step - loss: 0.2602 - accuracy: 0.8915 - val_loss: 0.3089 - val_accuracy: 0.8675
Epoch 4/5
214/214 [==============================] - 220s 1s/step - loss: 0.2058 - accuracy: 0.9123 - val_loss: 0.2931 - val_accuracy: 0.8783
Epoch 5/5
214/214 [==============================] - 220s 1s/step - loss: 0.1547 - accuracy: 0.9399 - val_loss: 0.3389 - val_accuracy: 0.8695


Validation Accuracy with Embedding size of 300 and RNN Units of 256 is 86.95

# Model 3- Glove Embedding of 300, With 128 Memory Size

In [43]:
model3 = tf.keras.Sequential()
model3.add(tf.keras.layers.Embedding(input_dim=vocab_size+1,
                  output_dim=300,
                  weights=[embedding_matrix],
                  trainable=False, mask_zero= True))
model3.add(tf.keras.layers.Dropout(0.2))
model3.add(tf.keras.layers.Bidirectional(LSTM(units=128,return_sequences=False),merge_mode='concat'))
model3.add(tf.keras.layers.Dense(1,activation='sigmoid'))
model3.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [44]:
model3.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 300)         8897100   
_________________________________________________________________
dropout_2 (Dropout)          (None, None, 300)         0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 256)               439296    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 9,336,653
Trainable params: 439,553
Non-trainable params: 8,897,100
_________________________________________________________________


In [45]:
model3.fit(X_train,y_train,
          epochs=15,
          batch_size=100, validation_data=(X_val, y_val))

Epoch 1/15
214/214 [==============================] - 221s 1s/step - loss: 0.4555 - accuracy: 0.7820 - val_loss: 0.3657 - val_accuracy: 0.8321
Epoch 2/15
214/214 [==============================] - 215s 1s/step - loss: 0.3342 - accuracy: 0.8538 - val_loss: 0.3265 - val_accuracy: 0.8568
Epoch 3/15
214/214 [==============================] - 218s 1s/step - loss: 0.2734 - accuracy: 0.8828 - val_loss: 0.3019 - val_accuracy: 0.8723
Epoch 4/15
214/214 [==============================] - 218s 1s/step - loss: 0.2246 - accuracy: 0.9078 - val_loss: 0.3075 - val_accuracy: 0.8712
Epoch 5/15
214/214 [==============================] - 217s 1s/step - loss: 0.1807 - accuracy: 0.9272 - val_loss: 0.3057 - val_accuracy: 0.8751
Epoch 6/15
214/214 [==============================] - 215s 1s/step - loss: 0.1384 - accuracy: 0.9460 - val_loss: 0.3160 - val_accuracy: 0.8819
Epoch 7/15
214/214 [==============================] - 215s 1s/step - loss: 0.1087 - accuracy: 0.9584 - val_loss: 0.3286 - val_accuracy: 0.8778

Validation Accuracy with Embedding size of 300 and RNN Units of 128 is 87.94